In [ ]:
%load_ext autoreload
%autoreload 2

from pystorm.hal import HAL
from pystorm.hal.net_builder import NetBuilder

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [ ]:
# test the D > 1 create_default_yx_taps function (with no bad taps)
# this is in the SY, SX space, not Y,X space

np.random.seed(1)

# making a Y-by-X pool of D dims located at (LY, LX)

Y, X = (8, 8)
LY, LX = (8, 0)

N = X * Y
D = 1

SY = Y // 2
SX = X // 2

In [ ]:
# this stuff should all work fine
hal = HAL()
net_builder = NetBuilder(hal)

# get the complete set of bad syn locations for the chip (slow PEs and high bias mismatch)
bad_syns, _ = net_builder.determine_bad_syns()

# create some nice taps, avoiding bad syns
syn_tap_matrix = NetBuilder.create_default_yx_taps(Y // 2, X // 2, D, 
                                                  bad_syn=bad_syns[LY // 2 : (LY + Y) // 2, 
                                                                   LX // 2 : (LX + X) // 2])
nrn_tap_matrix = NetBuilder.syn_taps_to_nrn_taps(syn_tap_matrix)
NetBuilder.make_taps_even(nrn_tap_matrix)

# set up the Network object
# CREATE THE NETWORK WITH BIAS = 3!
# we want as many people to fire as possible! otherwise we can't estimate gain
net = net_builder.create_single_pool_net(Y, X, loc_yx=(LY, LX), tap_matrix=nrn_tap_matrix, biases=3, gain_divs=1)
pool_obj = net.get_pools()[0]
input_obj = net.get_inputs()[0]
hal.map(net)

# determine fmax
safe_fmaxes = net_builder.determine_safe_fmaxes()
fmax = safe_fmaxes[pool_obj]

# this can screw up estimation
hal.set_DAC_value('DAC_SOMA_REF', 1024)

# estimate encoders and offsets
encoders, offsets, _, _ = net_builder.determine_encoders_and_offsets(pool_obj, input_obj, fmax)

# do it again, want to assess stability
encoders_again, offsets_again, _, _ = net_builder.determine_encoders_and_offsets(pool_obj, input_obj, fmax)

In [ ]:
# determine good bias twiddle settings based on calibration and measured offsets
bias_settings, new_offsets, good, bin_counts, dbg = \
    net_builder.optimize_bias_twiddles(pool_obj, encoders, offsets, policy='center')

In [ ]:
# plotting norm(encoders) vs offsets and vs new_offsets shows before-after effect of bias t

# the first one shows the two trials' results diff'd
NetBuilder.plot_neuron_yield_cone(encoders, offsets, good,
                                 (encoders_again, offsets_again, np.ones(offsets.shape, dtype=int) * 2))

# the second plot shows what the different bias levels should produce
NetBuilder.plot_neuron_yield_cone(encoders, offsets, good)
all_offsets = net_builder.get_pool_bias_twiddles(pool_obj)
ax = plt.gca()
for i in range(7):
    NetBuilder.plot_neuron_yield_cone(encoders, 
                                      offsets + all_offsets[i, :].flatten() - all_offsets[6, :].flatten(), 
                                      good, ax=ax)
    
    
NetBuilder.plot_neuron_yield_cone(encoders, new_offsets, good, (encoders, offsets, bias_settings))


In [ ]:
# now do the encoder estimation experiment again with these biases

net_builder = NetBuilder(hal)

net = net_builder.create_single_pool_net(
    Y, X, loc_yx=(LY, LX), tap_matrix=nrn_tap_matrix, biases=bias_settings, gain_divs=1)
pool_obj = net.get_pools()[0]
input_obj = net.get_inputs()[0]
hal.map(net)

encoders_ver, offsets_ver, _, _ = net_builder.determine_encoders_and_offsets(pool_obj, input_obj, fmax)

In [ ]:
NetBuilder.plot_neuron_yield_cone(encoders, offsets, good)

NetBuilder.plot_neuron_yield_cone(encoders, new_offsets, good, (encoders, offsets, bias_settings))

NetBuilder.plot_neuron_yield_cone(encoders_ver, offsets_ver, good, (encoders, offsets, bias_settings))

NetBuilder.plot_neuron_yield_cone(encoders_ver, offsets_ver, good, (encoders, new_offsets, bias_settings))

print(np.sum(good))
fired_orig = np.sum(~np.isnan(offsets))
fired_exp = np.sum(~np.isnan(new_offsets))
fired_ver = np.sum(~np.isnan(offsets_ver))
print('fired orig:', fired_orig)
print('fired exp:', fired_exp)
print('fired ver:', fired_ver)

good_orig = np.sum(NetBuilder.get_good_mask(encoders, offsets))
good_exp = np.sum(NetBuilder.get_good_mask(encoders, new_offsets))
good_ver = np.sum(NetBuilder.get_good_mask(encoders_ver, offsets_ver))
print('good orig:', good_orig)
print('good exp:', good_exp)
print('good ver:', good_ver)


In [ ]:
print(bias_settings)